In [1]:
from bs4 import BeautifulSoup as soup
from lxml import etree as ET
import pandas as pd
import unicodedata
import matplotlib.pyplot as plt 
import plotly.express as px
import numpy as np
import requests
import polars as pl

#from arcgis import GIS
import geopy
from geopy.geocoders import Nominatim

#from arcgis.features import SpatialDataFrame
#from arcgis.features import GeoAccessor
#from arcgis.gis import GIS

In [2]:
dump = pl.read_parquet('data/online_diss_cleaned_03-2024.parquet') 
data = dump.to_pandas()
#data = pd.read_csv('data/check-data-csv.csv')
data

,ID,Creator,Title,lang,Place,Year,DDC,DDC1,DDC2,DDC-DE?,Sachgruppe,diss,notes,008
0,1000000389,"Klöfkorn, Robert",Numerics for evolution equations : a general i...,eng,Freiburg (Breisgau),2009,N/A,N/A,N/A,510,N/A,"Freiburg (Breisgau), Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||eng
1,1000006395,"Speckmann, Bodo",Regulation der hepatischen Biosynthese von Sel...,ger,Düsseldorf,2009,570,570,DE-101,573.3845554,540,"Düsseldorf, Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||ger
2,100000645X,"Sauheitl, Leopold",Direct amino acid uptake by plants related to ...,eng,Bayreuth,2009,N/A,N/A,N/A,570,N/A,"Bayreuth, Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||eng
3,1000006948,"Sanchez, Martin",Charakterisierung des Replikationsenzyms ORF90...,ger,Bayreuth,2010,570,570,DE-101,572.86453629321,N/A,"Bayreuth, Univ., Diss., 2010",N/A,100204s2009 gw |||||om||| 00||||ger
4,1000007138,"Reimann, Bodo",Numerische Untersuchung der Stoßrohrströmung ...,ger,Duisburg,2009,620,620,DE-101,629.4152,620,"Duisburg, Essen, Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||ger
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324777,999997297,"Moschkau, Peter",Defektabhängige Transporteigenschaften von Pr...,ger,Göttingen,2009,N/A,N/A,N/A,530,N/A,"Göttingen, Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||ger
324778,999997505,"Scheler, Esther",Tailoring fluorene-based oligomers for micron ...,eng,Bayreuth,2009,540,540,DE-101,547.70455,VN 5927,"Bayreuth, Univ., Diss., 2009",N/A,100204s2009 gw |||||om||| 00||||eng
324779,99999767X,"Körzdörfer, Thomas",Self-interaction and charge transfer in organi...,eng,Bayreuth,2009,530,530,DE-101,537.6223,UM 1200,"Bayreuth, Univ., Diss., 2009",Enth. 4 Sonderabdr. aus verschiedenen Zeitschr.,100204s2009 gw |||||om||| 00||||eng
324780,999999389,"Xystrakis, Fotios",The drought tolerance limit of European beec...,eng,Freiburg (Breisgau),2009,N/A,N/A,N/A,630,N/A,"Freiburg (Breisgau), Univ., Diss., 2009",Dateien in unterschiedlichen Formaten,100204s2009 gw |||||om||| 00||||eng


In [3]:
data['diss'] = data['diss'].str.lower()

In [4]:
habil = data[data['diss'].str.contains("habil")]
#no_thesis = data[~data['diss'].str.contains("Diss")]
diss = data[data['diss'].str.contains("diss")]
master = data[data['diss'].str.contains("bachelor|master|magister|magisterarb")]
print("Habils: ", len(habil))
print("Dissen: ", len(diss))
print("Master: ", len(master))

Habils:  3341
Dissen:  321398
Master:  10


In [5]:
#find string in column: 
data[data['diss'].str.contains("bachel")]

,ID,Creator,Title,lang,Place,Year,DDC,DDC1,DDC2,DDC-DE?,Sachgruppe,diss,notes,008
103197,1134967705,"Bronnenmeyer, Thilo",Linear state space control of tethered flying ...,eng,Freiburg,2014,N/A,N/A,N/A,600,N/A,bachelorarbeit albert-ludwigs-universität fre...,IN COPYRIGHT http://rightsstatements.org/page/...,170609s2014 gw |||||om||| 00||||eng
198547,1253285683,"Schmidt, Paul Günter",Shaping climate-smart coffee landscapes,eng,Göttingen,2022,N/A,N/A,N/A,333.7,N/A,"bachelorarbeit göttingen, georg-august univer...",N/A,220311s2022 gw |||||om||| 00||||eng


In [6]:
#no_diss = data[~data['diss'].str.contains("diss")]
no_diss = data[data['diss'].str.contains("diss") == False]
print("No Diss: ", len(no_diss))
#no_habil_either = no_diss[~no_diss['diss'].str.contains("habil")]
no_habil_either = no_diss[no_diss['diss'].str.contains("habil") == False]
no_habil = diss[diss['diss'].str.contains("habil") == False]
print("No Diss and no Habil: ", len(no_habil_either))
print("No Habil: ", len(no_habil))

No Diss:  3384
No Diss and no Habil:  46
No Habil:  321395


In [7]:
both = data[data['diss'].str.contains("diss") & data['diss'].str.contains("habil")] 
#both

### Create Geodata:

In [8]:
uni = data[["ID", "diss"]]

In [9]:
uni[['place','uni','rest']]=uni['diss'].str.split(',', expand=True,n=2)
uni['place'] = uni['place'].str.replace("zugl.: ", "")
uni['place'] = uni['place'].str.replace("dissertation", "")

C:\Users\Steffi\AppData\Local\Temp\ipykernel_13284\3915493355.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni[['place','uni','rest']]=uni['diss'].str.split(',', expand=True,n=2)
C:\Users\Steffi\AppData\Local\Temp\ipykernel_13284\3915493355.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni[['place','uni','rest']]=uni['diss'].str.split(',', expand=True,n=2)
C:\Users\Steffi\AppData\Local\Temp\ipykernel_13284\3915493355.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [10]:
uni

,ID,diss,place,uni,rest
0,1000000389,"freiburg (breisgau), univ., diss., 2009",freiburg (breisgau),univ.,"diss., 2009"
1,1000006395,"düsseldorf, univ., diss., 2009",düsseldorf,univ.,"diss., 2009"
2,100000645X,"bayreuth, univ., diss., 2009",bayreuth,univ.,"diss., 2009"
3,1000006948,"bayreuth, univ., diss., 2010",bayreuth,univ.,"diss., 2010"
4,1000007138,"duisburg, essen, univ., diss., 2009",duisburg,essen,"univ., diss., 2009"
...,...,...,...,...,...
324777,999997297,"göttingen, univ., diss., 2009",göttingen,univ.,"diss., 2009"
324778,999997505,"bayreuth, univ., diss., 2009",bayreuth,univ.,"diss., 2009"
324779,99999767X,"bayreuth, univ., diss., 2009",bayreuth,univ.,"diss., 2009"
324780,999999389,"freiburg (breisgau), univ., diss., 2009",freiburg (breisgau),univ.,"diss., 2009"


In [11]:
uni.loc[((uni.place.str.contains('aachen'))), 'Hochschule'] = "Rheinisch-Westfälische Technische Hochschule Aachen"
uni.loc[((uni.place.str.contains('rheinisch'))), 'Hochschule'] = "Rheinisch-Westfälische Technische Hochschule Aachen"
uni.loc[((uni.place.str.contains('alfter'))), 'Hochschule'] = "Alanus Hochschule"
uni.loc[((uni.place.str.contains('köthen'))), 'Hochschule'] = "Hochschule Anhalt – Anhalt University of Applied Sciences"
uni.loc[((uni.place.str.contains('augsburg'))), 'Hochschule'] = "Universität Augsburg"
uni.loc[((uni.place.str.contains('bamberg'))), 'Hochschule'] = "Otto-Friedrich-Universität Bamberg"
uni.loc[((uni.place.str.contains('bayreuth'))), 'Hochschule'] = "Universität Bayreuth"
uni.loc[((uni.place.str.contains('berlin')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Freie Universität Berlin - likely"
uni.loc[((uni.place.str.contains('berlin')) & (uni.diss.str.contains('escp'))), 'Hochschule'] = "ESCP Europe Wirtschaftshochschule Berlin e.V."
uni.loc[((uni.place.str.contains('berlin')) & (uni.diss.str.contains('esmt'))), 'Hochschule'] = "ESMT European School of Management and Technology"
uni.loc[((uni.place.str.contains('berlin')) & (uni.diss.str.contains('freie'))), 'Hochschule'] = "Freie Universität Berlin"
uni.loc[((uni.place.str.contains('berlin')) & (uni.diss.str.contains('fu'))), 'Hochschule'] = "Freie Universität Berlin"
uni.loc[((uni.place.str.contains('berlin')) & (uni.diss.str.contains('fu'))), 'Hochschule'] = "Freie Universität Berlin"
uni.loc[((uni.place.str.contains('berlin')) & (uni.diss.str.contains('hertie'))), 'Hochschule'] = "Hertie School"
uni.loc[((uni.place.str.contains('berlin')) & (uni.diss.str.contains('hum'))), 'Hochschule'] = "Humboldt-Universität zu Berlin"
uni.loc[((uni.place.str.contains('berlin')) & (uni.diss.str.contains('hu'))), 'Hochschule'] = "Humboldt-Universität zu Berlin"
uni.loc[((uni.place.str.contains('berlin')) & (uni.diss.str.contains('tec'))), 'Hochschule'] = "Technische Universität Berlin"
uni.loc[((uni.place.str.contains('berlin')) & (uni.diss.str.contains('tu'))), 'Hochschule'] = "Technische Universität Berlin"
uni.loc[((uni.place.str.contains('berlin')) & (uni.diss.str.contains('künste'))), 'Hochschule'] = "Universität der Künste Berlin"
uni.loc[((uni.place.str.contains('berlin')) & (uni.diss.str.contains('ddr'))), 'Hochschule'] = "Akademie der Wissenschaften der DDR"
uni.loc[((uni.diss.str.contains('akademie')) & (uni.diss.str.contains('ddr'))), 'Hochschule'] = "Akademie der Wissenschaften der DDR"
uni.loc[((uni.place.str.contains('berlin')) & (uni.diss.str.contains('charit'))), 'Hochschule'] = "Charité – Universitätsmedizin Berlin"
uni.loc[((uni.place.str.contains('bielefeld'))), 'Hochschule'] = "Universität Bielefeld"
uni.loc[((uni.place.str.contains('bochum'))), 'Hochschule'] = "Ruhr-Universität Bochum"
uni.loc[((uni.place.str.contains('bocum'))), 'Hochschule'] = "Ruhr-Universität Bochum"
uni.loc[((uni.place.str.contains('boochum'))), 'Hochschule'] = "Ruhr-Universität Bochum"
uni.loc[((uni.place.str.contains('bochun'))), 'Hochschule'] = "Ruhr-Universität Bochum"
uni.loc[((uni.place.str.contains('bonn'))), 'Hochschule'] = "Rheinische Friedrich-Wilhelms-Universität Bonn"
uni.loc[((uni.place.str.contains('braunschweig')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Technische Universität Carolo-Wilhelmina zu Braunschweig"
uni.loc[((uni.place.str.contains('braunschweig')) & (uni.diss.str.contains('hoch'))), 'Hochschule'] = "Hochschule für Bildende Künste Braunschweig"
uni.loc[((uni.place.str.contains('braunschweig')) & (uni.diss.str.contains('tec'))), 'Hochschule'] = "Technische Universität Carolo-Wilhelmina zu Braunschweig"
uni.loc[((uni.place.str.contains('braunschweig')) & (uni.diss.str.contains('tu'))), 'Hochschule'] = "Technische Universität Carolo-Wilhelmina zu Braunschweig"
uni.loc[((uni.place.str.contains('bremen')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Universität Bremen"
uni.loc[((uni.place.str.contains('bremen')) & (uni.diss.str.contains('jac'))), 'Hochschule'] = "Jacobs University Bremen"
uni.loc[((uni.place.str.contains('chemnitz'))), 'Hochschule'] = "Technische Universität Chemnitz"
uni.loc[((uni.diss.str.contains('chemnitz'))), 'Hochschule'] = "Technische Universität Chemnitz"
uni.loc[((uni.place.str.contains('clausthal'))), 'Hochschule'] = "Technische Universität Clausthal"
uni.loc[((uni.place.str.contains('cottbus'))), 'Hochschule'] = "Brandenburgische Technische Universität Cottbus-Senftenberg"
uni.loc[((uni.place.str.contains('darmstadt')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Technische Universität Darmstadt"
uni.loc[((uni.place.str.contains('darmstadt')) & (uni.diss.str.contains('hoc'))), 'Hochschule'] = "Hochschule Darmstadt"
uni.loc[((uni.place.str.contains('darmstadt')) & (uni.diss.str.contains('tec'))), 'Hochschule'] = "Technische Universität Darmstadt"
uni.loc[((uni.place.str.contains('darmstadt')) & (uni.diss.str.contains('tu'))), 'Hochschule'] = "Technische Universität Darmstadt"
uni.loc[((uni.place.str.contains('darmstadt')) & (uni.diss.str.contains('th'))), 'Hochschule'] = "Hochschule Darmstadt"
uni.loc[((uni.place.str.contains('detmold'))), 'Hochschule'] = "Hochschule für Musik Detmold"
uni.loc[((uni.place.str.contains('dortmund'))), 'Hochschule'] = "Technische Universität Dortmund"
uni.loc[((uni.place.str.contains('dresden')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Technische Universität Dresden"
uni.loc[((uni.place.str.contains('dresden')) & (uni.diss.str.contains('künst'))), 'Hochschule'] = "Hochschule für Bildende Künste Dresden"
uni.loc[((uni.place.str.contains('dresden')) & (uni.diss.str.contains('musi'))), 'Hochschule'] = "Hochschule für Musik Carl Maria von Weber Dresden"
uni.loc[((uni.place.str.contains('dresden')) & (uni.diss.str.contains('tec'))), 'Hochschule'] = "Technische Universität Dresden"
uni.loc[((uni.place.str.contains('dresden')) & (uni.diss.str.contains('tu'))), 'Hochschule'] = "Technische Universität Dresden"
uni.loc[((uni.place.str.contains('dresden')) & (uni.diss.str.contains('tec'))), 'Hochschule'] = "Technische Universität Dresden"
uni.loc[((uni.diss.str.contains('dresden')) & (uni.diss.str.contains('tec'))), 'Hochschule'] = "Technische Universität Dresden"
uni.loc[((uni.diss.str.contains('dresden')) & (uni.diss.str.contains('tu'))), 'Hochschule'] = "Technische Universität Dresden"
uni.loc[((uni.place.str.contains('duisburg'))), 'Hochschule'] = "Universität Duisburg-Essen"
uni.loc[((uni.place.str.contains('essen'))), 'Hochschule'] = "Universität Duisburg-Essen"
uni.loc[((uni.place.str.contains('düsseldorf')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Heinrich-Heine-Universität Düsseldorf"
uni.loc[((uni.place.str.contains('düss')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Heinrich-Heine-Universität Düsseldorf"
uni.loc[((uni.place.str.contains('düsseldorf')) & (uni.diss.str.contains('hoc'))), 'Hochschule'] = "Robert-Schumann-Hochschule Düsseldorf"
uni.loc[((uni.place.str.contains('düsseldorf')) & (uni.diss.str.contains('kunst'))), 'Hochschule'] = "Kunstakademie Düsseldorf"
uni.loc[((uni.place.str.contains('düsseldorf')) & (uni.diss.str.contains('akad'))), 'Hochschule'] = "Kunstakademie Düsseldorf"
uni.loc[((uni.place.str.contains('eichstätt'))), 'Hochschule'] = "Katholische Universität Eichstätt-Ingolstadt"
uni.loc[((uni.place.str.contains('ingolstadt'))), 'Hochschule'] = "Katholische Universität Eichstätt-Ingolstadt"
uni.loc[((uni.place.str.contains('erfurt'))), 'Hochschule'] = "Universität Erfurt"
uni.loc[((uni.place.str.contains('erlangen'))), 'Hochschule'] = "Friedrich-Alexander-Universität Erlangen-Nürnberg"
uni.loc[((uni.place.str.contains('nürnberg'))), 'Hochschule'] = "Friedrich-Alexander-Universität Erlangen-Nürnberg"
uni.loc[((uni.place.str.contains('flensburg')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Europa-Universität Flensburg"
uni.loc[((uni.place.str.contains('flensburg')) & (uni.diss.str.contains('künste'))), 'Hochschule'] = "Folkwang Universität der Künste"
uni.loc[((uni.place.str.contains('flensburg')) & (uni.diss.str.contains('folkwang'))), 'Hochschule'] = "Folkwang Universität der Künste"
uni.loc[((uni.place.str.contains('flensburg')) & (uni.diss.str.contains('europa'))), 'Hochschule'] = "Europa-Universität Flensburg"
uni.loc[((uni.place.str.contains('flensburg')) & (uni.diss.str.contains('hoch'))), 'Hochschule'] = "Europa-Universität Flensburg"
uni.loc[((uni.place.str.contains('frankfurt')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Johann Wolfgang Goethe-Universität, Frankfurt am Main"
uni.loc[((uni.place.str.contains('frankfurt')) & (uni.diss.str.contains('main')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Johann Wolfgang Goethe-Universität, Frankfurt am Main"
uni.loc[((uni.place.str.contains('frankfurt')) & (uni.diss.str.contains('applied'))), 'Hochschule'] = "Frankfurt University of Applied Sciences"
uni.loc[((uni.place.str.contains('frankfurt')) & (uni.diss.str.contains('fin'))), 'Hochschule'] = "Frankfurt School of Finance & Management"
uni.loc[((uni.place.str.contains('frankfurt')) & (uni.diss.str.contains('musi'))), 'Hochschule'] = "Hochschule für Musik und Darstellende Kunst Frankfurt am Main"
uni.loc[((uni.place.str.contains('frankfurt')) & (uni.diss.str.contains('kunst'))), 'Hochschule'] = "Hochschule für Musik und Darstellende Kunst Frankfurt am Main"
uni.loc[((uni.place.str.contains('frankfurt')) & (uni.diss.str.contains('philo'))), 'Hochschule'] = "Philosophisch-Theologische Hochschule Sankt Georgen Frankfurt am Main"
uni.loc[((uni.place.str.contains('frankfurt')) & (uni.diss.str.contains('theo'))), 'Hochschule'] = "Philosophisch-Theologische Hochschule Sankt Georgen Frankfurt am Main"
uni.loc[((uni.place.str.contains('frankfurt')) & (uni.diss.str.contains('goethe'))), 'Hochschule'] = "Johann Wolfgang Goethe-Universität, Frankfurt am Main"
uni.loc[((uni.place.str.contains('frankfurt')) & (uni.diss.str.contains('oder'))), 'Hochschule'] = "Europa-Universität Viadrina Frankfurt (Oder)"
uni.loc[((uni.place.str.contains('frankfurt')) & (uni.diss.str.contains('viadrina'))), 'Hochschule'] = "Europa-Universität Viadrina Frankfurt (Oder)"
uni.loc[((uni.place.str.contains('freiburg')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Albert-Ludwigs-Universität Freiburg im Breisgau"
uni.loc[((uni.place.str.contains('freiberg'))), 'Hochschule'] = "Technische Universität Bergakademie Freiberg"
uni.loc[((uni.place.str.contains('freiburg')) & (uni.diss.str.contains('musi'))), 'Hochschule'] = "Hochschule für Musik Freiburg im Breisgau"
uni.loc[((uni.place.str.contains('freiburg')) & (uni.diss.str.contains('päd'))), 'Hochschule'] = "Pädagogische Hochschule Freiburg"
uni.loc[((uni.place.str.contains('freiburg')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Albert-Ludwigs-Universität Freiburg im Breisgau"
uni.loc[((uni.place.str.contains('friedrichshafen'))), 'Hochschule'] = "Zeppelin Universität – Hochschule zwischen Wirtschaft, Kultur und Politik"
uni.loc[((uni.place.str.contains('fulda')) & (uni.diss.str.contains('hoc'))), 'Hochschule'] = "Hochschule Fulda – University of Applied Sciences"
uni.loc[((uni.place.str.contains('fulda')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Hochschule Fulda – University of Applied Sciences"
uni.loc[((uni.place.str.contains('fulda')) & (uni.diss.str.contains('theo'))), 'Hochschule'] = "Theologische Fakultät Fulda"
uni.loc[((uni.place.str.contains('geisenheim'))), 'Hochschule'] = "Hochschule Geisenheim"
uni.loc[((uni.place.str.contains('gießen')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Justus-Liebig-Universität Gießen"
uni.loc[((uni.diss.str.contains('liebig')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Justus-Liebig-Universität Gießen"
uni.loc[((uni.place.str.contains('giessen')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Justus-Liebig-Universität Gießen"
uni.loc[((uni.place.str.contains('gießen')) & (uni.diss.str.contains('dvg'))), 'Hochschule'] = "Justus-Liebig-Universität Gießen"
uni.loc[((uni.place.str.contains('gießen')) & (uni.diss.str.contains('tec'))), 'Hochschule'] = "Technische Hochschule Mittelhessen – THM"
uni.loc[((uni.diss.str.contains('thm'))), 'Hochschule'] = "Technische Hochschule Mittelhessen – THM"
uni.loc[((uni.place.str.contains('göttingen'))), 'Hochschule'] = "Georg-August-Universität Göttingen"
uni.loc[((uni.place.str.contains('greifswald'))), 'Hochschule'] = "Universität Greifswald"
uni.loc[((uni.place.str.contains('hagen'))), 'Hochschule'] = "FernUniversität in Hagen"
uni.loc[((uni.place.str.contains('halle')) & (uni.diss.str.contains('kunst'))), 'Hochschule'] = "Burg Giebichenstein Kunsthochschule Halle"
uni.loc[((uni.place.str.contains('halle')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Martin-Luther-Universität Halle-Wittenberg"
uni.loc[((uni.place.str.contains('halle')) & (uni.diss.str.contains('witt'))), 'Hochschule'] = "Martin-Luther-Universität Halle-Wittenberg"
uni.loc[((uni.place.str.contains('hamburg')) & (uni.diss.str.contains('law'))), 'Hochschule'] = "Bucerius Law School"
uni.loc[((uni.diss.str.contains('bucerius'))), 'Hochschule'] = "Bucerius Law School"
uni.loc[((uni.place.str.contains('hamburg')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Universität Hamburg"
uni.loc[((uni.place.str.contains('hamburg')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Universität Hamburg"
uni.loc[((uni.place.str.contains('hamburg')) & (uni.diss.str.contains('wirtschaft'))), 'Hochschule'] = "Hamburger Universität für Wirtschaft und Politik"
uni.loc[((uni.place.str.contains('hamburg')) & (uni.diss.str.contains('hafen'))), 'Hochschule'] = "HafenCity Universität Hamburg"
uni.loc[((uni.place.str.contains('hamburg')) & (uni.diss.str.contains('bild'))), 'Hochschule'] = "Hochschule für Bildende Künste Hamburg"
uni.loc[((uni.place.str.contains('hamburg')) & (uni.diss.str.contains('künst'))), 'Hochschule'] = "Hochschule für Bildende Künste Hamburg"
uni.loc[((uni.place.str.contains('hamburg')) & (uni.diss.str.contains('musi'))), 'Hochschule'] = "Hochschule für Musik und Theater Hamburg"
uni.loc[((uni.place.str.contains('hamburg')) & (uni.diss.str.contains('thea'))), 'Hochschule'] = "Hochschule für Musik und Theater Hamburg"
uni.loc[((uni.place.str.contains('hamburg')) & (uni.diss.str.contains('musik'))), 'Hochschule'] = "Hochschule für Musik und Theater Hamburg"
uni.loc[((uni.place.str.contains('hamburg')) & (uni.diss.str.contains('kühne'))), 'Hochschule'] = "Kühne Logistics University – Wissenschaftliche Hochschule für Logistik und Unternehmensführung"
uni.loc[((uni.place.str.contains('hamburg')) & (uni.diss.str.contains('tec'))), 'Hochschule'] = "Technische Universität Hamburg"
uni.loc[((uni.place.str.contains('hamburg')) & (uni.diss.str.contains('tu'))), 'Hochschule'] = "Technische Universität Hamburg"
uni.loc[((uni.place.str.contains('hamburg')) & (uni.diss.str.contains('helmut'))), 'Hochschule'] = "Helmut-Schmidt-Universität/Universität der Bundeswehr Hamburg"
uni.loc[((uni.place.str.contains('hamburg')) & (uni.diss.str.contains('bundesw'))), 'Hochschule'] = "Helmut-Schmidt-Universität/Universität der Bundeswehr Hamburg"
uni.loc[((uni.place.str.contains('hannover')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Gottfried Wilhelm Leibniz Universität Hannover"
uni.loc[((uni.place.str.contains('hannover')) & (uni.diss.str.contains('unv'))), 'Hochschule'] = "Gottfried Wilhelm Leibniz Universität Hannover"
uni.loc[((uni.place.str.contains('hannover')) & (uni.diss.str.contains('musi'))), 'Hochschule'] = "Hochschule für Musik, Theater und Medien Hannover"
uni.loc[((uni.place.str.contains('hannover')) & (uni.diss.str.contains('thea'))), 'Hochschule'] = "Hochschule für Musik, Theater und Medien Hannover"
uni.loc[((uni.place.str.contains('hannover')) & (uni.diss.str.contains('medien'))), 'Hochschule'] = "Hochschule für Musik, Theater und Medien Hannover"
uni.loc[((uni.place.str.contains('hannover')) & (uni.diss.str.contains('mediz'))), 'Hochschule'] = "Medizinische Hochschule Hannover"
uni.loc[((uni.place.str.contains('hannov')) & (uni.diss.str.contains('med.'))), 'Hochschule'] = "Medizinische Hochschule Hannover"
uni.loc[((uni.place.str.contains('hannover')) & (uni.diss.str.contains('mhh'))), 'Hochschule'] = "Medizinische Hochschule Hannover"
uni.loc[((uni.diss.str.contains('hannover')) & (uni.diss.str.contains('tier'))), 'Hochschule'] = "Stiftung Tierärztliche Hochschule Hannover"
uni.loc[((uni.place.str.contains('heidelberg')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Ruprecht-Karls-Universität Heidelberg"
uni.loc[((uni.diss.str.contains('heidelberg')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Ruprecht-Karls-Universität Heidelberg"
uni.loc[((uni.place.str.contains('heidelberg')) & (uni.diss.str.contains('jüd'))), 'Hochschule'] = "Hochschule für Jüdische Studien Heidelberg"
uni.loc[((uni.place.str.contains('heidelberg')) & (uni.diss.str.contains('päd'))), 'Hochschule'] = "Pädagogische Hochschule Heidelberg"
uni.loc[((uni.place.str.contains('hildesheim'))), 'Hochschule'] = "Universität Hildesheim"
uni.loc[((uni.place.str.contains('hohenheim'))), 'Hochschule'] = "Universität Hohenheim"
uni.loc[((uni.place.str.contains('ilmenau'))), 'Hochschule'] = "Technische Universität Ilmenau"
uni.loc[((uni.place.str.contains('jena'))), 'Hochschule'] = "Friedrich-Schiller-Universität Jena"
uni.loc[((uni.place.str.contains('kaiserslautern'))), 'Hochschule'] = "Technische Universität Kaiserslautern"
uni.loc[((uni.diss.str.contains('karlsruhe')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Karlsruher Institut für Technologie"
uni.loc[((uni.place.str.contains('karlsruhe')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Karlsruher Institut für Technologie"
uni.loc[((uni.place.str.contains('karlsruhe')) & (uni.diss.str.contains('akad'))), 'Hochschule'] = "Staatliche Akademie der Bildenden Künste Karlsruhe"
uni.loc[((uni.place.str.contains('karlsruhe')) & (uni.diss.str.contains('künste'))), 'Hochschule'] = "Staatliche Akademie der Bildenden Künste Karlsruhe"
uni.loc[((uni.place.str.contains('karlsruhe')) & (uni.diss.str.contains('gest'))), 'Hochschule'] = "Staatliche Hochschule für Gestaltung Karlsruhe"
uni.loc[((uni.place.str.contains('karlsruhe')) & (uni.diss.str.contains('musi'))), 'Hochschule'] = "Hochschule für Musik Karlsruhe"
uni.loc[((uni.place.str.contains('karlsruhe')) & (uni.diss.str.contains('päd'))), 'Hochschule'] = "Pädagogische Hochschule Karlsruhe"
uni.loc[((uni.place.str.contains('karlsruhe')) & (uni.diss.str.contains('tec'))), 'Hochschule'] = "Karlsruher Institut für Technologie"
uni.loc[((uni.place.str.contains('karlsruher inst'))), 'Hochschule'] = "Karlsruher Institut für Technologie"
uni.loc[((uni.place.str.contains('kaiserslautern')) & (uni.diss.str.contains('landau'))), 'Hochschule'] = "Rheinland-Pfälzische Technische Universität Kaiserslautern-Landau"
uni.loc[((uni.place.str.contains('kit'))), 'Hochschule'] = "Karlsruher Institut für Technologie"
uni.loc[((uni.place.str.contains('karlsruhe')) & (uni.diss.str.contains('kit'))), 'Hochschule'] = "Karlsruher Institut für Technologie"
uni.loc[((uni.place.str.contains('kassel'))), 'Hochschule'] = "Universität Kassel"
uni.loc[((uni.place.str.contains('kiel')) & (uni.diss.str.contains('kunst'))), 'Hochschule'] = "Muthesius Kunsthochschule"
uni.loc[((uni.place.str.contains('kiel')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Christian-Albrechts-Universität zu Kiel"
uni.loc[((uni.place.str.contains('koblenz'))), 'Hochschule'] = "Universität Koblenz-Landau"
uni.loc[((uni.place.str.contains('landau'))), 'Hochschule'] = "Universität Koblenz-Landau"
uni.loc[((uni.place.str.contains('köln')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Universität zu Köln"
uni.loc[((uni.place.str.contains('köln')) & (uni.diss.str.contains('sport'))), 'Hochschule'] = "Deutsche Sporthochschule Köln"
uni.loc[((uni.place.str.contains('köln')) & (uni.diss.str.contains('hoch'))), 'Hochschule'] = "Deutsche Sporthochschule Köln"
uni.loc[((uni.place.str.contains('köln')) & (uni.diss.str.contains('musi'))), 'Hochschule'] = "Hochschule für Musik und Tanz Köln"
uni.loc[((uni.place.str.contains('köln')) & (uni.diss.str.contains('tanz'))), 'Hochschule'] = "Hochschule für Musik und Tanz Köln"
uni.loc[((uni.place.str.contains('köln')) & (uni.diss.str.contains('hoch'))), 'Hochschule'] = "Deutsche Sporthochschule Köln"
uni.loc[((uni.place.str.contains('köln')) & (uni.diss.str.contains('theo'))), 'Hochschule'] = "Kölner Hochschule für Katholische Theologie (KHKT)"
uni.loc[((uni.place.str.contains('khkt'))), 'Hochschule'] = "Kölner Hochschule für Katholische Theologie (KHKT)"
uni.loc[((uni.place.str.contains('augustin'))), 'Hochschule'] = "Philosophisch-Theologischen Hochschule SVD St. Augustin (Kölner Hochschule für Katholische Theologie (KHKT))"
uni.loc[((uni.place.str.contains('köln')) & (uni.diss.str.contains('kunst'))), 'Hochschule'] = "Kunsthochschule für Medien Köln"
uni.loc[((uni.place.str.contains('konstanz'))), 'Hochschule'] = "Universität Konstanz"
uni.loc[((uni.place.str.contains('leipzig')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Universität Leipzig"
uni.loc[((uni.place.str.contains('leipzig')) & (uni.diss.str.contains('hoch'))), 'Hochschule'] = "HHL Leipzig Graduate School of Management"
uni.loc[((uni.place.str.contains('leipzig')) & (uni.diss.str.contains('grafik'))), 'Hochschule'] = "Hochschule für Grafik und Buchkunst Leipzig"
uni.loc[((uni.place.str.contains('leipzig')) & (uni.diss.str.contains('buch'))), 'Hochschule'] = "Hochschule für Grafik und Buchkunst Leipzig"
uni.loc[((uni.place.str.contains('leipzig')) & (uni.diss.str.contains('hhl'))), 'Hochschule'] = "HHL Leipzig Graduate School of Management"
uni.loc[((uni.place.str.contains('leipzig')) & (uni.diss.str.contains('manage'))), 'Hochschule'] = "HHL Leipzig Graduate School of Management"
uni.loc[((uni.place.str.contains('leipzig')) & (uni.diss.str.contains('musi'))), 'Hochschule'] = "Hochschule für Musik und Theater Felix Mendelssohn Bartholdy Leipzig"
uni.loc[((uni.place.str.contains('leipzig')) & (uni.diss.str.contains('thea'))), 'Hochschule'] = "Hochschule für Musik und Theater Felix Mendelssohn Bartholdy Leipzig"
uni.loc[((uni.place.str.contains('lübeck')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Universität zu Lübeck"
uni.loc[((uni.place.str.contains('lübeck')) & (uni.diss.str.contains('musi'))), 'Hochschule'] = "Musikhochschule Lübeck"
uni.loc[((uni.place.str.contains('lübeck')) & (uni.diss.str.contains('hoc'))), 'Hochschule'] = "Musikhochschule Lübeck"
uni.loc[((uni.place.str.contains('ludwigsburg'))), 'Hochschule'] = "Pädagogische Hochschule Ludwigsburg"
uni.loc[((uni.place.str.contains('lüneburg'))), 'Hochschule'] = "Leuphana Universität Lüneburg"
uni.loc[((uni.diss.str.contains('magdeburg')) & (uni.diss.str.contains('hoc'))), 'Hochschule'] = "Hochschule Magdeburg-Stendal"
uni.loc[((uni.diss.str.contains('magdeburg')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Otto-von-Guericke-Universität Magdeburg"
uni.loc[((uni.diss.str.contains('magedeburg')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Otto-von-Guericke-Universität Magdeburg"
uni.loc[((uni.diss.str.contains('magdeburg')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Otto-von-Guericke-Universität Magdeburg"
uni.loc[((uni.place.str.contains('mainz'))), 'Hochschule'] = "Johannes Gutenberg-Universität Mainz"
uni.loc[((uni.place.str.contains('mannheim')) & (uni.diss.str.contains('hoc'))), 'Hochschule'] = "Staatliche Hochschule für Musik und Darstellende Kunst Mannheim"
uni.loc[((uni.place.str.contains('mannheim')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Universität Mannheim"
uni.loc[((uni.place.str.contains('mannheim')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Universität Mannheim"
uni.loc[((uni.place.str.contains('marburg'))), 'Hochschule'] = "Philipps-Universität Marburg"
uni.loc[((uni.place.str.contains('münchen')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Ludwig-Maximilians-Universität München"
uni.loc[((uni.place.str.contains('münchen')) & (uni.diss.str.contains('akad'))), 'Hochschule'] = "Akademie der Bildenden Künste München"
uni.loc[((uni.place.str.contains('münchen')) & (uni.diss.str.contains('künste'))), 'Hochschule'] = "Akademie der Bildenden Künste München"
uni.loc[((uni.place.str.contains('münchen')) & (uni.diss.str.contains('fern'))), 'Hochschule'] = "Hochschule für Fernsehen und Film München"
uni.loc[((uni.place.str.contains('münchen')) & (uni.diss.str.contains('film'))), 'Hochschule'] = "Hochschule für Fernsehen und Film München"
uni.loc[((uni.place.str.contains('münchen')) & (uni.diss.str.contains('musi'))), 'Hochschule'] = "Hochschule für Musik und Theater München"
uni.loc[((uni.place.str.contains('münchen')) & (uni.diss.str.contains('thea'))), 'Hochschule'] = "Hochschule für Musik und Theater München"
uni.loc[((uni.place.str.contains('münchen')) & (uni.diss.str.contains('philo'))), 'Hochschule'] = "Hochschule für Philosophie"
uni.loc[((uni.place.str.contains('münchen')) & (uni.diss.str.contains('tec'))), 'Hochschule'] = "Technische Universität München"
uni.loc[((uni.place.str.contains('münchen')) & (uni.diss.str.contains('tu'))), 'Hochschule'] = "Technische Universität München"
uni.loc[((uni.place.str.contains('munich')) & (uni.diss.str.contains('tec'))), 'Hochschule'] = "Technische Universität München"
uni.loc[((uni.place.str.contains('münchen')) & (uni.diss.str.contains('ludwig'))), 'Hochschule'] = "Ludwig-Maximilians-Universität München"
uni.loc[((uni.place.str.contains('münchen')) & (uni.diss.str.contains('lmu'))), 'Hochschule'] = "Ludwig-Maximilians-Universität München"
uni.loc[((uni.place.str.contains('münchen')) & (uni.diss.str.contains('bund'))), 'Hochschule'] = "Universität der Bundeswehr München"
uni.loc[((uni.place.str.contains('münchen')) & (uni.diss.str.contains('hoch'))), 'Hochschule'] = "Hochschule München"
uni.loc[((uni.place.str.contains('neubiberg')) & (uni.diss.str.contains('bund'))), 'Hochschule'] = "Universität der Bundeswehr München"
uni.loc[((uni.place.str.contains('münster')) & (uni.diss.str.contains('hoch'))), 'Hochschule'] = "Westfälische Wilhelms-Universität Münster"
uni.loc[((uni.place.str.contains('münster')) & (uni.diss.str.contains('kunst'))), 'Hochschule'] = "Kunstakademie Münster, Hochschule für Bildende Künste"
uni.loc[((uni.place.str.contains('münster')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Westfälische Wilhelms-Universität Münster"
uni.loc[((uni.place.str.contains('münster')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Westfälische Wilhelms-Universität Münster"
uni.loc[((uni.place.str.contains('neuendettelsau'))), 'Hochschule'] = "Augustana-Hochschule Neuendettelsau"
uni.loc[((uni.place.str.contains('polizei'))), 'Hochschule'] = "Deutsche Hochschule der Polizei"
uni.loc[((uni.place.str.contains('nürnberg')) & (uni.diss.str.contains('hoc'))), 'Hochschule'] = "Hochschule für Musik Nürnberg"
uni.loc[((uni.place.str.contains('nürnberg')) & (uni.diss.str.contains('musi'))), 'Hochschule'] = "Hochschule für Musik Nürnberg"
uni.loc[((uni.place.str.contains('offenbach'))), 'Hochschule'] = "Hochschule für Musik Nürnberg"
uni.loc[((uni.place.str.contains('oldenburg'))), 'Hochschule'] = "Carl von Ossietzky Universität Oldenburg"
uni.loc[((uni.place.str.contains('osnabrück'))), 'Hochschule'] = "Universität Osnabrück"
uni.loc[((uni.place.str.contains('paderborn')) & (uni.diss.str.contains('theol'))), 'Hochschule'] = "Theologische Fakultät Paderborn"
uni.loc[((uni.place.str.contains('paderborn')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Universität Paderborn"
uni.loc[((uni.place.str.contains('passau'))), 'Hochschule'] = "Universität Passau"
uni.loc[((uni.place.str.contains('potsdam')) & (uni.diss.str.contains('babel'))), 'Hochschule'] = "Filmuniversität Babelsberg Konrad Wolf"
uni.loc[((uni.place.str.contains('babelsberg'))), 'Hochschule'] = "Filmuniversität Babelsberg Konrad Wolf"
uni.loc[((uni.place.str.contains('potsdam')) & (uni.diss.str.contains('film'))), 'Hochschule'] = "Filmuniversität Babelsberg Konrad Wolf"
uni.loc[((uni.place.str.contains('potsdam')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Universität Potsdam"
uni.loc[((uni.place.str.contains('potsdam')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Universität Potsdam"
uni.loc[((uni.place.str.contains('wiesbaden'))), 'Hochschule'] = "Hochschule RheinMain"
uni.loc[((uni.place.str.contains('regensburg'))), 'Hochschule'] = "Universität Regensburg"
uni.loc[((uni.place.str.contains('regesnburg'))), 'Hochschule'] = "Universität Regensburg"
uni.loc[((uni.place.str.contains('rheinmain'))), 'Hochschule'] = "Hochschule RheinMain"
uni.loc[((uni.place.str.contains('rostock')) & (uni.diss.str.contains('hoc'))), 'Hochschule'] = "Hochschule für Musik und Theater Rostock"
uni.loc[((uni.place.str.contains('rostock')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Universität Rostock"
uni.loc[((uni.place.str.contains('saarbrücken')) & (uni.diss.str.contains('musi'))), 'Hochschule'] = "Hochschule für Musik Saar"
uni.loc[((uni.place.str.contains('saarbrücken')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Universität des Saarlandes"
uni.loc[((uni.place.str.contains('saar')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Universität des Saarlandes"
uni.loc[((uni.place.str.contains('homburg')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Universität des Saarlandes"
uni.loc[((uni.place.str.contains('saarbrücken')) & (uni.diss.str.contains('künste'))), 'Hochschule'] = "Hochschule der Bildenden Künste Saar"
uni.loc[((uni.place.str.contains('saar')) & (uni.diss.str.contains('künste'))), 'Hochschule'] = "Hochschule der Bildenden Künste Saar"
uni.loc[((uni.place.str.contains('schwäbisch gmünd'))), 'Hochschule'] = "Pädagogische Hochschule Schwäbisch Gmünd"
uni.loc[((uni.place.str.contains('siegen'))), 'Hochschule'] = "Universität Siegen"
uni.loc[((uni.place.str.contains('speyer'))), 'Hochschule'] = "Deutsche Universität für Verwaltungswissenschaften Speyer"
uni.loc[((uni.place.str.contains('stuttgart')) & (uni.diss.str.contains('akad'))), 'Hochschule'] = "Staatliche Akademie der Bildenden Künste Stuttgart"
uni.loc[((uni.place.str.contains('stuttgart')) & (uni.diss.str.contains('hoc'))), 'Hochschule'] = "Staatliche Hochschule für Musik und Darstellende Kunst Stuttgart"
uni.loc[((uni.place.str.contains('stuttgart')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Universität Stuttgart"
uni.loc[((uni.place.str.contains('trier')) & (uni.diss.str.contains('theo'))), 'Hochschule'] = "Theologische Fakultät Trier"
uni.loc[((uni.place.str.contains('trier')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Universität Trier"
uni.loc[((uni.place.str.contains('trossingen'))), 'Hochschule'] = "Staatliche Hochschule für Musik Trossingen"
uni.loc[((uni.place.str.contains('tübingen'))), 'Hochschule'] = "Eberhard Karls Universität Tübingen"
uni.loc[((uni.place.str.contains('ulm'))), 'Hochschule'] = "Universität Ulm"
uni.loc[((uni.place.str.contains('vallendar')) & (uni.diss.str.contains('vpu'))), 'Hochschule'] = "VPU Vinzenz Pallotti University"
uni.loc[((uni.place.str.contains('vallendar')) & (uni.diss.str.contains('phil'))), 'Hochschule'] = "VPU Vinzenz Pallotti University"
uni.loc[((uni.place.str.contains('vallendar')) & (uni.diss.str.contains('hoc'))), 'Hochschule'] = "VPU Vinzenz Pallotti University"
uni.loc[((uni.place.str.contains('vallendar')) & (uni.diss.str.contains('whu'))), 'Hochschule'] = "WHU – Otto Beisheim School of Management"
uni.loc[((uni.place.str.contains('vechta'))), 'Hochschule'] = "Universität Vechta"
uni.loc[((uni.place.str.contains('weimar')) & (uni.diss.str.contains('hoc'))), 'Hochschule'] = "Hochschule für Musik Franz Liszt Weimar"
uni.loc[((uni.place.str.contains('weimar')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Bauhaus-Universität Weimar"
uni.loc[((uni.place.str.contains('weingarten'))), 'Hochschule'] = "Pädagogische Hochschule Weingarten"
uni.loc[((uni.place.str.contains('wiesbaden')) & (uni.diss.str.contains('ebs'))), 'Hochschule'] = "EBS Universität für Wirtschaft und Recht"
uni.loc[((uni.place.str.contains('wiesbaden')) & (uni.diss.str.contains('business'))), 'Hochschule'] = "EBS Universität für Wirtschaft und Recht"
uni.loc[((uni.place.str.contains('oestrich-winkel')) & (uni.diss.str.contains('business'))), 'Hochschule'] = "EBS Universität für Wirtschaft und Recht"
uni.loc[((uni.place.str.contains('witten'))), 'Hochschule'] = "Private Universität Witten/Herdecke gGmbH"
uni.loc[((uni.place.str.contains('wuppertal')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Bergische Universität Wuppertal"
uni.loc[((uni.place.str.contains('wuppertal')) & (uni.diss.str.contains('hoch'))), 'Hochschule'] = "Kirchliche Hochschule Wuppertal/Bethel (Hochschule für Kirche und Diakonie)"
uni.loc[((uni.diss.str.contains('würzburg')) & (uni.diss.str.contains('uni'))), 'Hochschule'] = "Julius-Maximilians-Universität Würzburg"
uni.loc[((uni.diss.str.contains('würzburg')) & (uni.diss.str.contains('hoch'))), 'Hochschule'] = "Hochschule für Musik Würzburg"
uni.loc[((uni.place.str.contains('würzburg')) & (uni.diss.str.contains('hoch'))), 'Hochschule'] = "Hochschule für Musik Würzburg"


C:\Users\Steffi\AppData\Local\Temp\ipykernel_13284\877884117.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni.loc[((uni.place.str.contains('aachen'))), 'Hochschule'] = "Rheinisch-Westfälische Technische Hochschule Aachen"


In [12]:
uni["count"] = uni.groupby('Hochschule')['Hochschule'].transform('count')
uni

C:\Users\Steffi\AppData\Local\Temp\ipykernel_13284\2453902877.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni["count"] = uni.groupby('Hochschule')['Hochschule'].transform('count')


,ID,diss,place,uni,rest,Hochschule,count
0,1000000389,"freiburg (breisgau), univ., diss., 2009",freiburg (breisgau),univ.,"diss., 2009",Albert-Ludwigs-Universität Freiburg im Breisgau,7147
1,1000006395,"düsseldorf, univ., diss., 2009",düsseldorf,univ.,"diss., 2009",Heinrich-Heine-Universität Düsseldorf,7524
2,100000645X,"bayreuth, univ., diss., 2009",bayreuth,univ.,"diss., 2009",Universität Bayreuth,1876
3,1000006948,"bayreuth, univ., diss., 2010",bayreuth,univ.,"diss., 2010",Universität Bayreuth,1876
4,1000007138,"duisburg, essen, univ., diss., 2009",duisburg,essen,"univ., diss., 2009",Universität Duisburg-Essen,5071
...,...,...,...,...,...,...,...
324777,999997297,"göttingen, univ., diss., 2009",göttingen,univ.,"diss., 2009",Georg-August-Universität Göttingen,10454
324778,999997505,"bayreuth, univ., diss., 2009",bayreuth,univ.,"diss., 2009",Universität Bayreuth,1876
324779,99999767X,"bayreuth, univ., diss., 2009",bayreuth,univ.,"diss., 2009",Universität Bayreuth,1876
324780,999999389,"freiburg (breisgau), univ., diss., 2009",freiburg (breisgau),univ.,"diss., 2009",Albert-Ludwigs-Universität Freiburg im Breisgau,7147


In [13]:
uni.loc[uni['Hochschule'].isnull()]

,ID,diss,place,uni,rest,Hochschule,count


In [14]:
uni = uni.drop_duplicates(subset=['Hochschule'])
uni

,ID,diss,place,uni,rest,Hochschule,count
0,1000000389,"freiburg (breisgau), univ., diss., 2009",freiburg (breisgau),univ.,"diss., 2009",Albert-Ludwigs-Universität Freiburg im Breisgau,7147
1,1000006395,"düsseldorf, univ., diss., 2009",düsseldorf,univ.,"diss., 2009",Heinrich-Heine-Universität Düsseldorf,7524
2,100000645X,"bayreuth, univ., diss., 2009",bayreuth,univ.,"diss., 2009",Universität Bayreuth,1876
4,1000007138,"duisburg, essen, univ., diss., 2009",duisburg,essen,"univ., diss., 2009",Universität Duisburg-Essen,5071
5,1000007820,"greifswald, univ., diss., 2009",greifswald,univ.,"diss., 2009",Universität Greifswald,3044
...,...,...,...,...,...,...,...
222994,1293538396,"dissertation rostock, hochschule für musik un...",rostock,hochschule für musik und theater rostock 2022,None,Hochschule für Musik und Theater Rostock,3
225030,1293855820,dissertation filmuniversität babelsberg konra...,filmuniversität babelsberg konrad wolf 2021,None,None,Filmuniversität Babelsberg Konrad Wolf,4
238550,1322059616,"dissertation alfter, alanus hochschule 2023",alfter,alanus hochschule 2023,None,Alanus Hochschule,1
240583,957743157,"hamburg, hochsch. für wirtschaft und politik,...",hamburg,hochsch. für wirtschaft und politik,"diss., 1999",Hamburger Universität für Wirtschaft und Politik,18


In [15]:
## Connect to Geolocator:
geolocator = Nominatim(timeout=10, user_agent = "myGeolocator")

C:\Users\Steffi\anaconda3\Lib\site-packages\geopy\adapters.py:668: UserWarning: urllib3 prior to 1.24.2 is known to have a bug with custom ssl contexts: it attempts to load system certificates to them. Please consider upgrading `requests` and `urllib3` packages. See https://github.com/urllib3/urllib3/pull/1566
  warnings.warn(


In [16]:
#Add new colum 'gcode' that contains translated Ort-value to geolocation: 
#WARNING: THIS MAY TAKE A WHILE

uni['gcode'] = uni.Hochschule.apply(geolocator.geocode)
uni

C:\Users\Steffi\AppData\Local\Temp\ipykernel_13284\1149198745.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni['gcode'] = uni.Hochschule.apply(geolocator.geocode)


,ID,diss,place,uni,rest,Hochschule,count,gcode
0,1000000389,"freiburg (breisgau), univ., diss., 2009",freiburg (breisgau),univ.,"diss., 2009",Albert-Ludwigs-Universität Freiburg im Breisgau,7147,"(Albert-Ludwigs-Universität Freiburg, Münsterp..."
1,1000006395,"düsseldorf, univ., diss., 2009",düsseldorf,univ.,"diss., 2009",Heinrich-Heine-Universität Düsseldorf,7524,"(Heinrich-Heine-Universität Düsseldorf, 1, Uni..."
2,100000645X,"bayreuth, univ., diss., 2009",bayreuth,univ.,"diss., 2009",Universität Bayreuth,1876,"(Universität Bayreuth, 30, Universitätsstraße,..."
4,1000007138,"duisburg, essen, univ., diss., 2009",duisburg,essen,"univ., diss., 2009",Universität Duisburg-Essen,5071,"(Universität Duisburg-Essen, Südviertel, Stadt..."
5,1000007820,"greifswald, univ., diss., 2009",greifswald,univ.,"diss., 2009",Universität Greifswald,3044,"(Universität Greifswald, Samtens, West-Rügen, ..."
...,...,...,...,...,...,...,...,...
222994,1293538396,"dissertation rostock, hochschule für musik un...",rostock,hochschule für musik und theater rostock 2022,None,Hochschule für Musik und Theater Rostock,3,"(Hochschule für Musik und Theater, 8, Beim St...."
225030,1293855820,dissertation filmuniversität babelsberg konra...,filmuniversität babelsberg konrad wolf 2021,None,None,Filmuniversität Babelsberg Konrad Wolf,4,"(Filmuniversität Babelsberg „Konrad Wolf“, 11,..."
238550,1322059616,"dissertation alfter, alanus hochschule 2023",alfter,alanus hochschule 2023,None,Alanus Hochschule,1,"(Alanus Hochschule, Olsdorfer Heide, Olsdorf, ..."
240583,957743157,"hamburg, hochsch. für wirtschaft und politik,...",hamburg,hochsch. für wirtschaft und politik,"diss., 1999",Hamburger Universität für Wirtschaft und Politik,18,None


In [17]:
dfmissmatch = uni[uni['gcode'].isnull()]
dfmissmatch

,ID,diss,place,uni,rest,Hochschule,count,gcode
41,1000016145,"würzburg, univ., diss., 2009",würzburg,univ.,"diss., 2009",Julius-Maximilians-Universität Würzburg,8736,None
2685,100140095X,"zugl.: oestrich-winkel, europ. business school...",oestrich-winkel,europ. business school,", diss., 2009",EBS Universität für Wirtschaft und Recht,6,None
6679,1005409846,"hannover, univ., diss., 2010",hannover,univ.,"diss., 2010",Gottfried Wilhelm Leibniz Universität Hannover,3126,None
8711,1007581883,"sankt augustin, philos.-theol. hochsch., diss....",sankt augustin,philos.-theol. hochsch.,"diss., 2009",Philosophisch-Theologischen Hochschule SVD St....,6,None
11301,100950438X,"frankfurt (oder), univ., diss., 2010",frankfurt (oder),univ.,"diss., 2010",Europa-Universität Viadrina Frankfurt (Oder),145,None
12261,1009923501,"zugl.: braunschweig, techn. univ., diss., 2010",braunschweig,techn. univ.,"diss., 2010",Technische Universität Carolo-Wilhelmina zu Br...,3154,None
13668,1010641611,"zugl.: vallendar, philos.-theol. hochsch., dis...",vallendar,philos.-theol. hochsch.,"diss., 2010",VPU Vinzenz Pallotti University,41,None
21185,1017244324,"berlin, univ., diss., 2011",berlin,univ.,"diss., 2011",Freie Universität Berlin - likely,6,None
38437,1031199470,"zugl.: witten / herdecke, univ., diss., 2012",witten / herdecke,univ.,"diss., 2012",Private Universität Witten/Herdecke gGmbH,46,None
56402,1053311974,"wuppertal/bethel, kirchliche hochsch., diss., ...",wuppertal/bethel,kirchliche hochsch.,"diss., 2014",Kirchliche Hochschule Wuppertal/Bethel (Hochsc...,2,None


In [18]:
location = geolocator.geocode("Julius-Maximilians-Universität Würzburg")
print(location)

None


In [19]:
##

dfmissmatch.to_csv("data/uni_missmatch.csv")

In [79]:
new_gnd = pd.read_csv("data/uni_missmatch_gnd.csv", sep=";", usecols = ['ID', 'GND', 'GND-ID'])
#new_gnd

In [80]:
matched = dfmissmatch.merge(new_gnd, on='ID', indicator=True)
matched

,ID,diss,place,uni,rest,Hochschule,count,gcode,GND,GND-ID,_merge
0,1000016145,"würzburg, univ., diss., 2009",würzburg,univ.,"diss., 2009",Julius-Maximilians-Universität Würzburg,8736,None,Julius-Maximilians-Universität Würzburgzburg,36189-6,both
1,100140095X,"zugl.: oestrich-winkel, europ. business school...",oestrich-winkel,europ. business school,", diss., 2009",EBS Universität für Wirtschaft und Recht,6,None,EBS Universität für Wirtschaft und Recht,1032822171,both
2,1005409846,"hannover, univ., diss., 2010",hannover,univ.,"diss., 2010",Gottfried Wilhelm Leibniz Universität Hannover,3126,None,Gottfried Wilhelm Leibniz Universität Hannover,64681-7,both
3,1007581883,"sankt augustin, philos.-theol. hochsch., diss....",sankt augustin,philos.-theol. hochsch.,"diss., 2009",Philosophisch-Theologischen Hochschule SVD St....,6,None,Philosophisch-Theologische Hochschule SVD,213851-7,both
4,100950438X,"frankfurt (oder), univ., diss., 2010",frankfurt (oder),univ.,"diss., 2010",Europa-Universität Viadrina Frankfurt (Oder),145,None,Europa-Universität Viadrina Frankfurt (Oder),5059953-7,both
5,1009923501,"zugl.: braunschweig, techn. univ., diss., 2010",braunschweig,techn. univ.,"diss., 2010",Technische Universität Carolo-Wilhelmina zu Br...,3154,None,Technische Universität Braunschweig,36227-X,both
6,1010641611,"zugl.: vallendar, philos.-theol. hochsch., dis...",vallendar,philos.-theol. hochsch.,"diss., 2010",VPU Vinzenz Pallotti University,41,None,Vinzenz Pallotti University,1247200272,both
7,1017244324,"berlin, univ., diss., 2011",berlin,univ.,"diss., 2011",Freie Universität Berlin - likely,6,None,Freie Universitat Berlin,2114189-7,both
8,1031199470,"zugl.: witten / herdecke, univ., diss., 2012",witten / herdecke,univ.,"diss., 2012",Private Universität Witten/Herdecke gGmbH,46,None,Universität Witten-Herdecke,511081-6,both
9,1053311974,"wuppertal/bethel, kirchliche hochsch., diss., ...",wuppertal/bethel,kirchliche hochsch.,"diss., 2014",Kirchliche Hochschule Wuppertal/Bethel (Hochsc...,2,None,Kirchliche Hochschule Wuppertal/Bethel,10158862-8,both


In [81]:
query_ids = matched['GND-ID'].to_list()
query_ids

['36189-6',
 '1032822171',
 '64681-7',
 '213851-7',
 '5059953-7',
 '36227-X',
 '1247200272',
 '2114189-7',
 '511081-6',
 '10158862-8',
 '16344287-3',
 '40680-6',
 '5286092-9',
 '16102609-6',
 '10032988-3',
 '81500-7']

In [139]:
## Jetzt noch Geokoordinaten über SRU der GND abfragen und hinzufügen: 

def gnb_sru(gndid):
    
    base_url = "https://services.dnb.de/sru/authorities"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': 'nid="' + str(gndid) + '"'
         }
    
    r = requests.get(base_url, params=params)
    xml = soup(r.content, features="xml")
    records = xml.find_all('record', {'type':'Authority'})
    #print("Treffer: ", len(records))
    treffer = xml.find_all("numberOfRecords")[0].text
    #print("number of records: ", treffer)
    
    print("GND-ID: ", gndid)
    
    if len(records) == 1: 
        
        #print("Es geht weiter")
        for record in records:
            content = record
    
        ns = {"marc":"http://www.loc.gov/MARC21/slim"}
        xml1 = ET.fromstring(unicodedata.normalize("NFC", str(content)))
        
        #get place first: 
        place1 = xml1.xpath("marc:datafield[@tag = '510']", namespaces=ns)
        place2 = xml1.xpath("marc:datafield[@tag = '551']", namespaces=ns)
        #place2 = xml1.xpath("marc:datafield[@tag = '551']/marc:subfield[@code = '0']", namespaces=ns)
        #rela2 = xml1.xpath("marc:datafield[@tag = '551']/marc:subfield[@code = '4']", namespaces=ns)
        
        # Search for datafields with tag 510 or 551
        places = xml1.xpath('marc:datafield[@tag="510" or @tag="551"]', namespaces=ns)

        for datafield in places:
        # Find subfields with code 4
            subfields_4 = datafield.findall('marc:subfield[@code="4"]', namespaces=ns)

            # Check if any subfield with code 4 contains "orta"
            for subfield_4 in subfields_4:
                if "orta" in subfield_4.text:
                    # Find the first subfield with code 0 in the same datafield
                    subfield_0 = datafield.find('marc:subfield[@code="0"]', namespaces=ns)

                    if subfield_0 is not None:
                        place = subfield_0.text
                        place = place.split(")", 1)[1]
        
        
        
        
        print(place)
            
        params.update({'query': 'nid="' + place + '"'})
        request_place = requests.get(base_url, params=params)
        print(request_place.url)
        
        xml2 = soup(request_place.content, features="xml")
        more_records = xml2.find_all('record', {'type':'Authority'})
        treffer2 = xml2.find_all("numberOfRecords")[0].text
        print(treffer2)
        
        
        for new_record in more_records: 
        
            xml3 = ET.fromstring(unicodedata.normalize("NFC", str(new_record)))
            long = xml3.xpath("marc:datafield[@tag = '034']/marc:subfield[@code = 'd']", namespaces=ns)
            lat = xml3.xpath("marc:datafield[@tag = '034']/marc:subfield[@code = 'f']", namespaces=ns)
            for elem in long:
                if "." in elem.text: 
                    long = elem.text
                    if long.startswith("E"):
                        long = long.replace("E", "")
                        long = long.lstrip('0') 
                    elif long.startswith("W"):
                        long = long.replace("W", "")
                        long = long.lstrip('0') 
                        long = "-" + long
                        #print(long)
            for elem in lat:
                if "." in elem.text: 
                    lat = elem.text
                    if lat.startswith("N"):
                        lat = lat.replace("N", "")
                        lat = lat.lstrip('0') 
                    elif lat.startswith("S"):
                        lat = lat.replace("S", "")
                        lat = lat.lstrip('0') 
                        lat = "-" + lat
                        #print(lat)
        
            coordinates = {'GND-ID': gndid, 'lat': lat, 'long': long}
            return coordinates
             
    else:
        
            print("ERROR - ", gndid)
            coordinates = {'GND-ID': gndid, 'lat': ' ', 'long': ' '}
            return coordinates
        

In [140]:
myquery = [gnb_sru(queryid) for queryid in query_ids]

GND-ID:  36189-6
040670376
https://services.dnb.de/sru/authorities?recordSchema=MARC21-xml&operation=searchRetrieve&version=1.1&maximumRecords=100&query=nid%3D%22040670376%22
0
GND-ID:  1032822171
040660435
https://services.dnb.de/sru/authorities?recordSchema=MARC21-xml&operation=searchRetrieve&version=1.1&maximumRecords=100&query=nid%3D%22040660435%22
0
GND-ID:  64681-7
040233499
https://services.dnb.de/sru/authorities?recordSchema=MARC21-xml&operation=searchRetrieve&version=1.1&maximumRecords=100&query=nid%3D%22040233499%22
0
GND-ID:  213851-7
041053168
https://services.dnb.de/sru/authorities?recordSchema=MARC21-xml&operation=searchRetrieve&version=1.1&maximumRecords=100&query=nid%3D%22041053168%22
0
GND-ID:  5059953-7
040181227
https://services.dnb.de/sru/authorities?recordSchema=MARC21-xml&operation=searchRetrieve&version=1.1&maximumRecords=100&query=nid%3D%22040181227%22
0
GND-ID:  36227-X
04008065X
https://services.dnb.de/sru/authorities?recordSchema=MARC21-xml&operation=searchRe

In [132]:
print(myquery)

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [84]:
df2 = pd.DataFrame(myquery)
df2

AttributeError: 'NoneType' object has no attribute 'keys'

In [ ]:
uni.to_json("data/unidata.json")

In [ ]:
df = pd.read_json("data/unidata.json")
df